In [1]:
# Importamos JSON
import json

# Guardamos las credenciales para consumir api de twitter (sacadas de apps.twiiter.com)
credentials = {
    "KEY": "c0Jlf7B9Ez56i6rsnBSeiBCii",
    "KEY_SECRET": "yuCAV2TdGrvitWBGQondppKaxNlQfnrc5krRCYvNpu3miY2gQQ",
    "TOKEN": "1279067386520440833-YLZvyZzPDZlQ3bGa5SH2fhugVPl4Cs",
    "ACCESS_SECRET": "DZuy6rzCJFZ4723jVl2gITgf2jCXr1cqlFRtQXFytrIKD"
}

credentials_PRO = {
    "KEY": "XqNe5tbaycDxrrIOVqHR1NoPk",
    "KEY_SECRET": "wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q",
    "TOKEN": "1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ",
    "ACCESS_SECRET": "rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM"
}

credentials
credentials_PRO

{'KEY': 'XqNe5tbaycDxrrIOVqHR1NoPk',
 'KEY_SECRET': 'wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q',
 'TOKEN': '1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ',
 'ACCESS_SECRET': 'rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM'}

In [2]:
# Intslamos Twythom
!pip install twython

In [6]:
# Instalamos geopy
!pip install geopy

In [5]:
#importamos las librerias necesarias
import pandas as pd

### consultar api tiempo real
from twython import TwythonStreamer

### Se guardara en un archivo .csv
import csv

## imortar libreria para buscar coordenadas
from geopy.geocoders import Nominatim



In [69]:
## EJEMPLO de como encontrra las coordenadas sabiendo el pais y/o ciudad
geolocator = Nominatim(user_agent="BD-Project-JJ")

coordinates =  []

location = geolocator.geocode("Port Moody, British Columbia")

if location:
    coordinates.append(location.latitude)
    coordinates.append(location.longitude)
else:
    no_location += 1
    coordinates.append(0)
    coordinates.append(0)
coordinates
        

[49.2822243, -122.8293424]

In [60]:
geolocator = Nominatim(user_agent="BD-Project-JJ")


#procesar los datos con filtro
def process_tweet(tweet):
    # titulos
    #date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,retweet,quote,reply,likes,isquote,originaldate,originaltext,originaluser,originalverified,originallocation,originalquotecount,originalreplycount,originalretweetcount,originalfavoritecount,language,device,location,latitude,longitude

        d = {}
        d['date'] = tweet['created_at']
        d["hashtags"] = [hashtag["text"] for hashtag in tweet['entities']['hashtags']]
        d['text'] = tweet['text']
        #datos user
        d['user'] = tweet['user']['screen_name']
        d['verified'] = tweet['user']['verified']
        d['created'] = tweet['user']['created_at']
        d['followers'] = tweet['user']['followers_count']
        d['following'] = tweet['user']['friends_count']
        d['tweets'] = tweet['user']['statuses_count']
        d['Favorites'] = tweet['user']['favourites_count']
        
        # datos tweet
        d['retweet'] = tweet['retweet_count']
        d['quote'] = tweet['quote_count']
        d['reply'] = tweet['reply_count'] 
        d['like'] = tweet['favorite_count']

        # Si tiene un tweet original
        d['isQuote'] = tweet['is_quote_status']
        if tweet['is_quote_status']:
            d['originalTweetDate'] = tweet['quoted_status']['created_at']
            d['originalTweetText'] = tweet['quoted_status']['text']
            d['originalTweetUser'] = tweet['quoted_status']['user']['screen_name']
            d['originalTweetVerified'] = tweet['quoted_status']['user']['verified']
            d['originalTweetLocation'] = tweet['quoted_status']['user']['location']
            d['originalTweetQuote'] = tweet['quoted_status']['quote_count']
            d['originalTweetReply'] = tweet['quoted_status']['reply_count']
            d['originalTweetRetweet'] = tweet['quoted_status']['retweet_count']
            d['originalTweetFavorite'] = tweet['quoted_status']['favorite_count']
        else:
            d['originalTweetDate'] = "None"
            d['originalTweetText'] = "None"
            d['originalTweetUser'] = "None"
            d['originalTweetVerified'] = "None"
            d['originalTweetLocation'] = "None"
            d['originalTweetQuote'] = "None"
            d['originalTweetReply'] = "None"
            d['originalTweetRetweet'] = "None"
            d['originalTweetFavorite'] = "None"
            
                
        d['language'] = tweet['lang']

        ## guardar dispositivo
        if "android" in tweet['source']:
            d['device'] = "ANDROID"
        elif "iphone" in tweet['source']:
            d['device'] = "IPHONE"
        elif "ipad" in tweet['source']:
            d['device'] = "IPAD"
        elif "Web App" in tweet['source']:
            d['device'] = "WEB APP"
        else:
            d['device'] = "OTHER"


        d['user_location'] = tweet['user']['location']
        ## guardar coordenadas ubicacion
        if tweet['user']['location']:
            coordinates = search_coordinates(tweet['user']['location'])
            d['latitude'] = coordinates[0]
            d['longitude'] = coordinates[1]
        else:
            d['latitude'] = "None"
            d['longitude'] = "None"


        return d

def search_coordinates(location):
    #Buscar y guardar coordenadas
    coordinates = []

    location = geolocator.geocode(location)

    if location:
        coordinates.append(location.latitude)
        coordinates.append(location.longitude)
    else:
        coordinates.append("None")
        coordinates.append("None")
    return coordinates

# Heredando clase
#se le pasa las credenciales (probar siempre con credentials normales.)
class Streamer(TwythonStreamer):

    #Datos recibidos
    def on_success(self, data):
        #si se necesita poner un condicional
        #if data['atributo'] == 'condicion':
        #print(data)
        tweet_data = process_tweet(data)
        self.save_tweet(tweet_data)

    #Error
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    #Guardar en un archivo el resultados
    def save_tweet(self, tweet):
        #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
        #with open(r'tweets.csv', 'a', encoding="utf-8") as file: 
        with open(r'/home/jovyan/work/slides/data/tweets.csv', 'a', encoding="utf-8") as file:
            write = csv.writer(file)
            write.writerow(list(tweet.values()))

In [5]:
### Recoleccion de datos

In [61]:

# Intanciar
stream =  Streamer(credentials['KEY'], credentials['KEY_SECRET'], credentials['TOKEN'], credentials['ACCESS_SECRET'])
#stream =  Streamer(credentials_PRO['KEY'], credentials_PRO['KEY_SECRET'], credentials_PRO['TOKEN'], credentials_PRO['ACCESS_SECRET'])

#Iniciar consulta en tiempo real
stream.statuses.filter(track="covid")
# Para poder para la consulta y luego ver los resultados debe parar el proceso manualmente, es decir interrumpirlo.
# dando click en el boton de interrupt IPython kernel

KeyError: 'created_at'

In [62]:
tweets = pd.read_csv("/home/jovyan/work/slides/data/tweets.csv")
tweets

,date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,...,originallocation,originalquotecount,originalreplycount,originalretweetcount,originalfavoritecount,language,device,location,latitude,longitude
0,Wed Jul 08 23:33:31 +0000 2020,[],RT @hey_itsNDO: This isn’t the flu. The scary ...,_fabeany_,False,Thu Mar 21 01:46:23 +0000 2019,303,641,25860,45871,...,Around the world,665,288,1637,2123,en,ANDROID,cis she/her,None,None
1,Wed Jul 08 23:33:31 +0000 2020,[],"RT @ActualidadRT: El presidente de Brasil, Jai...",burmosA,False,Thu Jul 08 05:31:48 +0000 2010,608,1078,59524,7435,...,None,None,None,None,None,es,ANDROID,Tabasco México,17.95078635,-92.48312213241397
2,Wed Jul 08 23:33:31 +0000 2020,[],RT @drakes_d4: Sanitization and delivery/take ...,badgyalrhorz,False,Fri Oct 04 13:41:49 +0000 2019,34,34,1992,94,...,"Chicago, IL",2040,197,42,153,en,IPHONE,NaN,None,None
3,Wed Jul 08 23:33:31 +0000 2020,[],RT @ddiamond: TONY FAUCI effectively agrees wi...,TheRealFMCH,False,Wed Jul 05 15:22:44 +0000 2017,217,1116,12175,10816,...,None,None,None,None,None,en,ANDROID,The Universe,55.041152350000004,9.810374515557207
4,Wed Jul 08 23:33:31 +0000 2020,[],@realDonaldTrump https://t.co/dty3tttVDe,Tim17430322,False,Tue Apr 21 20:52:52 +0000 2020,0,0,426,30,...,None,None,None,None,None,und,WEB APP,Here,43.8222157,17.6896937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Wed Jul 08 23:33:34 +0000 2020,[],El Ayuntamiento redobla esfuerzos para cuidar ...,VisionPe,False,Tue Apr 19 15:44:01 +0000 2011,986,1043,127711,83,...,None,None,None,None,None,es,OTHER,NaN,None,None
162,Wed Jul 08 23:33:34 +0000 2020,[],RT @klillington: My view on the app. Little ev...,Maura2e2_,False,Tue Jun 14 12:12:32 +0000 2016,3264,4997,73829,75704,...,None,None,None,None,None,en,IPAD,"Co Clare, Ireland",52.857257450000006,-8.937435925994537
163,Wed Jul 08 23:33:34 +0000 2020,['Arizona'],RT @Cleavon_MD: I’m an ER Doctor. I’m not send...,Rebecca74222325,False,Sat Jun 03 20:55:53 +0000 2017,0,9,39,222,...,None,None,None,None,None,en,WEB APP,NaN,None,None
164,Wed Jul 08 23:33:34 +0000 2020,[],RT @Phoo_NoLimit: I will be sending $200 to th...,lulsoniaaa,False,Tue Mar 17 10:56:58 +0000 2020,8,37,31,7,...,None,None,None,None,None,en,IPHONE,NaN,None,None


In [63]:
import numpy as np

rows_no_coordinates = np.where(tweets['latitude']=="None")[0]

tweets_with_coordinates = tweets.drop(rows_no_coordinates)

tweets_with_coordinates

,date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,...,originallocation,originalquotecount,originalreplycount,originalretweetcount,originalfavoritecount,language,device,location,latitude,longitude
1,Wed Jul 08 23:33:31 +0000 2020,[],"RT @ActualidadRT: El presidente de Brasil, Jai...",burmosA,False,Thu Jul 08 05:31:48 +0000 2010,608,1078,59524,7435,...,None,None,None,None,None,es,ANDROID,Tabasco México,17.95078635,-92.48312213241397
3,Wed Jul 08 23:33:31 +0000 2020,[],RT @ddiamond: TONY FAUCI effectively agrees wi...,TheRealFMCH,False,Wed Jul 05 15:22:44 +0000 2017,217,1116,12175,10816,...,None,None,None,None,None,en,ANDROID,The Universe,55.041152350000004,9.810374515557207
4,Wed Jul 08 23:33:31 +0000 2020,[],@realDonaldTrump https://t.co/dty3tttVDe,Tim17430322,False,Tue Apr 21 20:52:52 +0000 2020,0,0,426,30,...,None,None,None,None,None,und,WEB APP,Here,43.8222157,17.6896937
5,Wed Jul 08 23:33:31 +0000 2020,[],.@ManUtd @Mike_Phelan_1 snap him up I beg,ReeceMUnitedV7,False,Fri May 29 16:16:16 +0000 2020,201,237,3434,4659,...,NaN,22,14,7,97,en,ANDROID,Yer da's head,50.8250684,-0.1761844
6,Wed Jul 08 23:33:31 +0000 2020,[],RT @mhdksafa: If you're wondering why the COVI...,iTrishLane,False,Fri Jan 31 18:28:43 +0000 2014,686,731,56433,14777,...,None,None,None,None,None,en,IPHONE,Earth,34.2331373,-102.4107493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Wed Jul 08 23:33:34 +0000 2020,[],RT @_timos_: Should We Experimentally Infect P...,LQVentures,False,Wed Nov 14 22:43:47 +0000 2012,258,322,78237,4,...,None,None,None,None,None,en,OTHER,Global,43.7283874,-79.34914879325001
157,Wed Jul 08 23:33:34 +0000 2020,['FAKE'],RT @jornalextra: É #FAKE que edição da revista...,BrandaoGueiros,False,Fri Mar 09 17:24:42 +0000 2018,21154,20236,136420,118355,...,None,None,None,None,None,pt,ANDROID,London,51.5073219,-0.1276474
159,Wed Jul 08 23:33:34 +0000 2020,[],Arizona Supreme Court getting more pressure to...,anneryman,True,Mon May 11 19:49:13 +0000 2009,6214,3325,14316,4793,...,None,None,None,None,None,en,OTHER,"Phoenix, Ariz.",33.4484367,-112.0741417
162,Wed Jul 08 23:33:34 +0000 2020,[],RT @klillington: My view on the app. Little ev...,Maura2e2_,False,Tue Jun 14 12:12:32 +0000 2016,3264,4997,73829,75704,...,None,None,None,None,None,en,IPAD,"Co Clare, Ireland",52.857257450000006,-8.937435925994537


In [64]:
tweets_with_coordinates.to_csv("/home/jovyan/work/slides/data/tweetCoordinates.csv",  index = False, header=True)

In [65]:
# Intslamos gmplot
!pip install gmplot

In [66]:
latitude = pd.to_numeric(tweets_with_coordinates["latitude"], downcast="float")
longitude = pd.to_numeric(tweets_with_coordinates["longitude"], downcast="float")

latitude

1      17.950787
3      55.041153
4      43.822216
5      50.825069
6      34.233139
         ...    
156    43.728386
157    51.507320
159    33.448437
162    52.857258
165    38.254238
Name: latitude, Length: 101, dtype: float32

In [68]:
import gmplot

# instancia
gmap = gmplot.GoogleMapPlotter(30,0,3)

#insertar puntos en el mapa pasando una lista de coordenadas
gmap.heatmap(latitude.tolist(), longitude.to_list(), radius=10)


#Guardar el mapa
gmap.draw('work/slides/heatmap.html')